<a href="https://colab.research.google.com/github/Jcasttrop/MLOpsCourse/blob/main/Homework_1_MLOps_Zoomcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error


In [2]:
jan_data = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='pyarrow')
feb_data = pd.read_parquet('yellow_tripdata_2023-02.parquet', engine='pyarrow')

In [3]:
#Question 1
jan_data.shape

(3066766, 19)

In [4]:
#Question 2


#Total trip time
jan_data['tpep_pickup_datetime'] = pd.to_datetime(jan_data['tpep_pickup_datetime'])
jan_data['tpep_dropoff_datetime'] = pd.to_datetime(jan_data['tpep_dropoff_datetime'])

jan_data['total_time'] = (jan_data['tpep_dropoff_datetime'] - jan_data['tpep_pickup_datetime']).dt.total_seconds() / 60
jan_data['total_time'].std()

42.59435124195458

In [5]:
#Question 3
#Outlier removal

filtered_df = jan_data[(jan_data['total_time'] >= 1) & (jan_data['total_time'] <= 60)]
fraction_remaining = len(filtered_df) / len(jan_data)

print(f"Fraction of records left after dropping outliers: {fraction_remaining:.2f}")

Fraction of records left after dropping outliers: 0.98


In [6]:
#Question 4
#One hot encoding

relevant_columns = ['VendorID','passenger_count','trip_distance','PULocationID','DOLocationID','payment_type','fare_amount','extra','total_amount']
train_data = jan_data[relevant_columns].fillna(0)

# Convert location IDs to strings (important for dictionary vectorizer)
train_data["PULocationID"] = train_data["PULocationID"].astype(str)
train_data["DOLocationID"] = train_data["DOLocationID"].astype(str)
train_data['VendorID'] = train_data['VendorID'].astype(str)

# Create a list of dictionaries from the DataFrame (one dict per row)
data_dict = train_data.to_dict(orient='records')

# Fit a dictionary vectorizer
vectorizer = DictVectorizer()
vectorizer.fit(data_dict)

# Get the feature matrix (one-hot encoded)
feature_matrix = vectorizer.transform(data_dict)

# Calculate the dimensionality (number of columns)
dimensionality = feature_matrix.shape[1]

# Print the dimensionality
print(f"Dimensionality of the feature matrix: {dimensionality}")

Dimensionality of the feature matrix: 526


In [7]:
#Question 5


# Train a linear regression model with default parameters
model = LinearRegression()
model.fit(train_data, jan_data["fare_amount"])  # Assuming "fare_amount" is the target

# Calculate the RMSE on the training data
y_pred = model.predict(train_data)
rmse = mean_squared_error(jan_data["fare_amount"], y_pred, squared=False)  # Calculate RMSE (square root of MSE)

# Print the RMSE on train
print(f"RMSE on train: {rmse}")  # Format to two decimal places

RMSE on train: 3.5787408930271074e-14


In [9]:
#Question 6

# Select relevant columns for validation data (same as training data)
relevant_columns = ['VendorID','passenger_count','trip_distance','PULocationID','DOLocationID','payment_type','fare_amount','extra','total_amount']

# Impute missing values with 0 (use the same strategy as training)
feb_data_processed = feb_data[relevant_columns].fillna(0)

# Convert location IDs and VendorID to strings (important for consistency)
feb_data_processed["PULocationID"] = feb_data_processed["PULocationID"].astype(str)
feb_data_processed["DOLocationID"] = feb_data_processed["DOLocationID"].astype(str)
feb_data_processed['VendorID'] = feb_data_processed['VendorID'].astype(str)

# Transform the validation data using the fitted vectorizer
data_dict = feb_data_processed.to_dict(orient='records')

# Fit a dictionary vectorizer
vectorizer = DictVectorizer()
vectorizer.fit(data_dict)

# Get the feature matrix (one-hot encoded)
feature_matrix = vectorizer.transform(data_dict)



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 528 features, but LinearRegression is expecting 9 features as input.